# Third-party software computation model example

## Introduction

This example file demonstrates how to build a RunModel object which enables UQpy to execute models in third-party software. The files necessary to run this example are:
1. The input template - 'abaqus_input.py'
2. The model script - 'abaqus_fire_analysis.py'
3. The output script - 'extract_abaqus_output.py'
4. The script used by the output script - 'abaqus_output_script.py'

Note: To execute the example in this notebook, it is necessary to have access to the finite element solver Abaqus. 

## Details of the model

This example builds and analyzes a finite element model of a beam bearing uniformly distributed load, which is then subjected to fire load.  

## The script:

Import the python modules used in this example, note down the start time and the current directory, which will be used later to save the results.

In [1]:
from UQpy.Reliability import SubsetSimulation
import matplotlib.pyplot as plt
from UQpy.SampleMethods import MCMC
from UQpy.Distributions import Distribution
from UQpy.RunModel import RunModel
import glob
import pickle
import os
import math
import time

calling_directory = os.getcwd()
t = time.time()

### Building the model:

There are two probabilistic input variables, the fire load density and the yield strength. The fire load density is denoted as 'qtd' and the yield strength is denoted as 'fy' in the template input script. These are different from the default variable names used by RunModel, and hence they must be passed in as one of the inputs while building the RunModel object.

In [2]:
var_names = ['qtd', 'fy']

#### Create the model object

In [3]:
abaqus_sfe_model = RunModel(model_script='abaqus_fire_analysis_subset_sfe.py',
                            input_template='abaqus_input_subset_sfe.py',
                            output_script='extract_abaqus_output_subset_sfe.py',
                            var_names=['qtd', 'fy'], model_dir='Subset_SFE')
print('Example: Created the model object.')

Example: Created the model object.


### Create the MCMC object

The subset simulation algorithm operates in the standard normal space. So, the input template to the RunModel object has been modified to accept samples from standard normal random variables and transform it to have the desired distribution. The MCMC object needs the target distribution and proposal distribution to be specified, which is done next.

In [4]:
# Specify the target distribution. This is standard normal for use with subset simulation in UQpy.
dist = Distribution(dist_name=['normal', 'normal'], params=[[0, 1], [0, 1]])
# The next command specifies the proposal density. It is passed in as a list of distributions, one each for each dimension of
# of the random vector for use with the modified Metropolis-Hastings sampling algorithm.
dist_prop = [Distribution('normal', params=[0., 1]), Distribution('normal', params=[0., 1])]
mcmc_object = MCMC(dimension=2, algorithm='MMH', log_pdf_target=dist.log_pdf, proposal=dist_prop)

### Perform subset simulation

In [ ]:
x_ss = SubsetSimulation(MCMC_object=mcmc_object, RunModel_object=abaqus_sfe_model, 
                        samples_init=None, p_cond=0.1, nsamples_ss=10, verbose=True)

UQpy: Running Subset Simulation with MMH....
Out:  0
Example: Successful output extraction.
Out:  0
Example: Successful output extraction.
Out:  0
Example: Successful output extraction.
Out:  0
Example: Successful output extraction.


### Save the results

In [ ]:
results = {'pf': x_ss.pf, 'cov': x_ss.cov1, 'samples': x_ss.samples, 'qois': x_ss.RunModel_object.qoi_list}

# Pickle the results dictionary in the current directory. The basename and extension of the desired pickle file:
res_basename = 'Subset_sfe_results'
res_extension = '.pkl'

# Create a new results file with a larger index than any existing results files with the same name in the current
# directory.
res_file_list = glob.glob(res_basename + '_???' + res_extension)
if len(res_file_list) == 0:
    res_file_name = res_basename + '_000' + res_extension
else:
    max_number = max(res_file_list).split('.')[0].split('_')[-1]
    res_file_name = res_basename + '_%03d' % (int(max_number) + 1) + res_extension

res_file_name = os.path.join(calling_directory, res_file_name)
# Save the results to this new file.
with open(res_file_name, 'wb') as f:
    pickle.dump(results, f)
print('Saved the results to ' + res_file_name)

### Plot the results

In [ ]:
# Load the results from the latest pickle dump.
with open(res_file_name, 'rb') as f:
    results = pickle.load(f)

colors = ['red', 'blue', 'green', 'cyan', 'magenta', 'yellow', 'black']
# Display the results
pf = results['pf']
cov = results['cov']
print('Probability of failure: ', pf)
print('Cov: ', cov)

qois = results['qois']
print(len(qois))

samples = results['samples']
# Plot the results
for i in range(len(samples)):
    print('Len: ', len(samples[i][:, 0]))
    print(samples[i])
    plt.scatter(samples[i][:, 0], samples[i][:, 1], color=colors[i], marker='o')
plt.xlabel('$\mathregular{U_1}$')
plt.ylabel('$\mathregular{U_2}$')
plt.show()


# Transform the random variable
def phi(x=0):
    return (1.0 + math.erf(x / math.sqrt(2.0))) / 2.0


# Plot the results - samples converted to values used
for i in range(len(samples)):
    q = [50 + 400 * phi(x) for x in samples[i][:, 0]]
    f = samples[i][:, 1] * 0.07 * 250e6 + 250e6
    plt.scatter(q, f, color=colors[i], marker='o')
plt.xlabel('$\mathregular{q_{td}}$')
plt.ylabel('$\mathregular{f_y}$')
plt.show()

print('Example: Done!')
print('Time elapsed: %.2f minutes' % float((time.time() - t) / 60.0))